In [15]:
import os
import sys
os.chdir('C:\\Users\\carpe\\Betting_models')

In [16]:
import csv
import math
import numpy as np
import ast 
from scipy.stats import poisson 

In [17]:
MyData = 'Icehockey_OU_data_3000.csv' # extracting the data we have been provided by Tipico Quant team.

teamHOCKEY_list = [] # We create an empty list. We will use this list to add all the name of the teams we are dealing with.

teams_performance = open('teamHOCKEY_list.txt', 'w') # We create a text file where we will deposit the performance statas for each team. 
teams_performance.write("""{""")            

1

In [18]:
# we want to iterate over our data file and find all the team names that are going to be used. 
# We open our csv file, skipping the first line (titles).
MyDataRead = csv.reader(open(MyData)) 
next(MyDataRead)                                    
                                   
for row in MyDataRead: # We create a list of all the ice hockey teams.
                    
    if row[2] not in teamHOCKEY_list:
        teamHOCKEY_list.append(row[2]) 
                                 
    if row[4] not in teamHOCKEY_list:
        teamHOCKEY_list.append(row[4])

teamHOCKEY_list.sort() # We order alphabetically our team list .


In [19]:
# We create a dynamic table where keeping the record of: local goals, foreign goals, local conceded goals, foreign conceded goals.
# In addition, we track the amount of local and foreign matches.
for team in teamHOCKEY_list:
    teams_performance.write("""	'%s': {'local_goals': 0, 'foreign_goals': 0, 'local_goals_conceded': 0, 'foreign_goals_conceded': 0, 
    'local_games': 0, 'foreign_games': 0, 'Attack_Ratio_local': 0, 'Defend_Ratio_local': 0, 'Attack_Ratio_foreign': 0, 'Defend_Ratio_foreign': 0},
    """ % (team))

#  We write the end of the file and close it. This is so as to save what we have written if we want to open it again.
teams_performance.write("}") 
teams_performance.close()

# We create our dictionary where we will hold and update our data.
# We use the ast module which will read the teamHOCKEY_list.txt file and then create a dictionary called ‘dict’.
getting_current_performance = open('teamHOCKEY_list.txt', 'r').read() 
dict = ast.literal_eval(getting_current_performance)

In [20]:
# open another iteration of the data and skip the first line as usual, before we start with our main loop.
MyDataRead = csv.reader(open(MyData))
next(MyDataRead)
Probabilities= {}
odds = {}

In [21]:
# we look at each individual game in our data file and do some work on each of them.
# we want to reset these after every game we have analysed.
for game in MyDataRead:
    local_team = game[2] # We get the name of the local team
    foreign_team = game[4] # We get the name of the foreign team
    local_goals = int(game[19]) # We get the number goals of the local team
    foreign_goals = int(game[20]) # We get the number goals of the foreign team
    #print(local_team,':',local_goals, foreign_team,':',foreign_goals)
    
    curr_local_goals = 0
    curr_foreign_goals = 0
    avg_local_goals = 1
    avg_foreign_goals = 1
   
    # We update the Performance Stats for both involved teams..
    for key, value in dict.items():
        curr_local_goals += dict[key]['local_goals']
        curr_foreign_goals += dict[key]['foreign_goals']
        
        #print(curr_local_goals,curr_foreign_goals)
        
        if dict[key]['local_games'] >1  and dict[key]['foreign_games'] >1: 
            avg_local_goals = curr_local_goals / (dict[key]['local_games'] + dict[key]['foreign_games'])
            avg_foreign_goals = curr_foreign_goals / (dict[key]['local_games'] + dict[key]['foreign_games'])
           
    
    # We calculate 4 ratios to approach the current potential performance of the team in Attacking/Defending
    if dict[key]['local_games'] >1  and dict[key]['foreign_games'] >1:
            local_team_attack = (dict[local_team]['Attack_Ratio_local'] + dict[local_team]['Attack_Ratio_foreign']) / 2
            foreign_team_attack = (dict[foreign_team]['Attack_Ratio_local'] + dict[foreign_team]['Attack_Ratio_foreign']) / 2
 
            local_team_defend = (dict[local_team]['Defend_Ratio_local'] + dict[local_team]['Defend_Ratio_foreign']) / 2
            foreign_team_defend = (dict[foreign_team]['Defend_Ratio_local'] + dict[foreign_team]['Defend_Ratio_foreign']) / 2
 
            # expected value for how many goals will be scored by both the local and the foreign team.
            local_team_exp = avg_local_goals * local_team_attack * foreign_team_defend
            foreign_team_exp = avg_foreign_goals * foreign_team_attack * local_team_defend
            
       
    # Probability calculation using cummulative distribution function of Poisson.      
            
            # Probability of the event: under 4.5 goals.
            under4_5 = poisson.cdf(4, (local_team_exp+foreign_team_exp)/2) 
            
            
            # Probability of the event: over 4.5 goals.
            over4_5 = 1 - under4_5 # By definition
            
            
            # Probability of the even: under 6.5 goals.
            under6_5 = poisson.cdf(6, (local_team_exp+foreign_team_exp)/2) # here we have under 6.5
                      
            # Probability of the event: over 6.5 goals
            over6_5 = 1 - under6_5 # By definition
            
            g = dict[key]['local_games'] + dict[key]['foreign_games']      
            
            Probabilities[(team, g)] = {"under4_5" : under4_5, "over4_5" : over4_5, "under6_5" : under6_5, "over6_5" : over6_5} 
            # We store the probabilities per team AND the number of games they have played         
            
            # Fair odds calculation:
            
            oddUnder4_5 = 1/under4_5
            oddOver4_5 = 1/over4_5
            
            oddUnder6_5 = 1/under6_5
            oddOver4_5 = 1/over4_5
            
            
                    
           
                           
             # UPDATE VARIABLES AFTER MATCH HAS BEEN PLAYED
             # adding the goals scored and conceded in this game we are in right now, and also the total tally of games played.
    dict[local_team]['local_goals'] += local_goals
    dict[local_team]['local_goals_conceded'] += foreign_goals
    dict[local_team]['local_games'] += 1
 
    dict[foreign_team]['foreign_goals'] += foreign_goals
    dict[foreign_team]['foreign_goals_conceded'] += local_goals
    dict[foreign_team]['foreign_games'] += 1
   
    
            
    # We create the current performance team ratios
    for key, value in dict.items():
        if dict[key]['local_games'] >1  and dict[key]['foreign_games'] >1:
            Attack_Ratio_local = (dict[key]['local_goals'] / dict[key]['local_games']) / avg_local_goals
            Defend_Ratio_local = (dict[key]['local_goals_conceded'] / dict[key]['local_games']) / avg_foreign_goals
 
            Attack_Ratio_foreign = (dict[key]['foreign_goals'] / dict[key]['foreign_games']) / avg_foreign_goals
            Defend_Ratio_foreign = (dict[key]['foreign_goals_conceded'] / dict[key]['foreign_games']) / avg_local_goals
 
            dict[key]['Attack_Ratio_local'] = Attack_Ratio_local
            dict[key]['Defend_Ratio_local'] = Defend_Ratio_local
            dict[key]['Attack_Ratio_foreign'] = Attack_Ratio_foreign
            dict[key]['Defend_Ratio_foreign'] = Defend_Ratio_foreign

C:\Users\carpe\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\carpe\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in double_scalars
